In [1]:
# %%capture
# !pip install mediapipe
# !pip install mat4py
# !pip install xgboost

In [1]:
file_path = 'AFLW2000/'

### 1.import libraries

In [2]:
import os
import numpy as np
from random import randint
import tensorflow as tf
from PIL import Image, ImageFilter
from math import cos, sin
import glob
from pathlib import Path
import pandas as pd
%pylab inline
import matplotlib.pyplot as plt
import scipy
import os,cv2,math,glob,random
import scipy.io as sio
from sklearn.model_selection import train_test_split
from mat4py import loadmat
import mediapipe
import mediapipe as mp
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,accuracy_score,r2_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error,mean_absolute_error
import pickle 

Populating the interactive namespace from numpy and matplotlib


C:\Users\dell\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['cos', 'sin', 'randint']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## 2.Load Data

In [5]:
def load_mat_files(file_path):
    os.chdir(file_path)

# Get a list for .mat files in current folder
    mat_files = glob.glob('*.mat')

# List for stroring all the data
    all_data = []

# Iterate mat files
    for fname in mat_files:
    # Load mat file data into data.
        data = loadmat(fname)

        # Append data to the list
        all_data.append(data)
    return all_data , mat_files


In [6]:
AllMatData , MatFileNames = load_mat_files(file_path)

In [7]:
def get_file_name(mat_files):
    file_names = []
    for f in mat_files:
        file_names.append(f[0:len(f)-4])
    return file_names 


file_names = get_file_name(MatFileNames)

### 3.get Face points by mediapip

In [8]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles

In [9]:
def get_points_train():
    x_points = []
    y_points = []
    files_name = []

    faceModule = mediapipe.solutions.face_mesh
  # loading image and its correspinding mat file
    with faceModule.FaceMesh(static_image_mode=True) as faces:
        # loading the image
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

            for file_name in file_names:
                image = cv2.imread('../AFLW2000/'+file_name+'.jpg')

                # processing the face to extract the landmark points (468 point) for each x,y,z
                results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                if results.multi_face_landmarks != None:
                    # looping over the faces in the image
                    for face in results.multi_face_landmarks:
                        x_list=[]
                        y_list=[]
                        for landmark in face.landmark:
                            x = landmark.x
                            y = landmark.y

                    # note: the x and y values are scaled to the their width and height so we will get back their actual value in the image
                            shape = image.shape 
                            relative_x = int(x * shape[1])
                            relative_y = int(y * shape[0])

                            # cv2.putText(image, str(relative_y), (int(relative_x),int(relative_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,255,0), 2)
                            cv2.circle(image, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)
                            x_list.append(relative_x)
                            y_list.append(relative_y)
                        x_points.append(x_list)
                        y_points.append(y_list) 
                        files_name.append(file_name)

    return x_points , y_points , files_name


In [10]:
x_points , y_points , files_name = get_points_train()


### 4.normalize points

In [11]:
def normalize_train(x_points , y_points):
    x_points = np.array(x_points)
    y_points = np.array(y_points)
    Center_X = x_points - x_points[:,99].reshape(-1,1)
    Center_Y = y_points - y_points[:,99].reshape(-1,1)

    X_171 = x_points[:,152]
    X_10 = x_points[:,10]
    Y_171 = y_points[:,152]
    Y_10 = y_points[:,10]
    distance = np.linalg.norm(np.array((X_10,Y_10)) - np.array((X_171,Y_171)),axis = 0).reshape(-1,1)
    Norm_X = Center_X / distance
    Norm_Y = Center_Y / distance
    
    return Norm_X , Norm_Y


In [12]:
Norm_X , Norm_Y = normalize_train(x_points , y_points)

### 5.get pitch yaw roll

In [13]:
def load_available(files):
    all_data = []
    names = []
    for fname in files:
    # Load mat file data into data.
        data = loadmat(fname+'.mat')

    # Append data to the list
        all_data.append(data)
        names.append(fname)

    return all_data , names


In [14]:
ava_data , ava_mat_files = load_available(files_name)

In [17]:
def get_pyr(ava_mat_files , files_name):
    ctr = 0
    roll = []
    pitch = []
    yaw = []
    for i in range(len(files_name)):
        if files_name[i] == ava_mat_files[i]:
            pitch.append(ava_data[i]['Pose_Para'][0])
            yaw.append(ava_data[i]['Pose_Para'][1])
            roll.append(ava_data[i]['Pose_Para'][2])
            ctr+=1
    print(ctr)
    return pitch, yaw, roll


In [18]:
pitch, yaw, roll = get_pyr(ava_mat_files , files_name)

1853


### 6.concat x , y , yaw , pitch , roll

In [21]:
data_x = pd.DataFrame(Norm_X)
data_x

,0,1,2,3,4,5,6,7,8,9,...,458,459,460,461,462,463,464,465,466,467
0,0.041736,0.050083,0.050083,0.029215,0.050083,0.058430,0.075124,-0.187810,0.083471,0.087645,...,0.087645,0.108512,0.146074,0.079298,0.075124,0.179463,0.158595,0.146074,0.379794,0.400661
1,0.000000,-0.067483,0.018405,-0.012270,-0.067483,-0.036809,0.061349,0.061349,0.092023,0.085888,...,0.000000,0.012270,0.110427,-0.012270,0.012270,0.226990,0.196315,0.165641,0.398766,0.423305
2,0.058812,0.068614,0.058812,0.019604,0.068614,0.068614,0.049010,-0.254853,0.039208,0.039208,...,0.098020,0.117624,0.147031,0.098020,0.088218,0.127426,0.107822,0.098020,0.303863,0.313665
3,0.078045,0.004878,0.048778,-0.053656,-0.014633,-0.024389,-0.034145,-0.234135,-0.068289,-0.087801,...,0.058534,0.073167,0.141457,0.053656,0.058534,0.087801,0.063412,0.048778,0.282913,0.297547
4,0.065819,0.030715,0.043879,-0.017552,0.021940,0.013164,-0.004388,-0.232559,-0.017552,-0.026327,...,0.070207,0.083370,0.127249,0.065819,0.061431,0.087758,0.065819,0.057043,0.250111,0.267662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,0.042347,0.079989,0.051757,0.037642,0.084694,0.079989,0.056463,-0.282313,0.051757,0.056463,...,0.103515,0.122336,0.131746,0.098810,0.084694,0.117630,0.098810,0.098810,0.287018,0.301134
1849,0.043980,0.087959,0.057173,0.048377,0.092357,0.092357,0.070367,-0.241887,0.070367,0.074765,...,0.105551,0.127541,0.131939,0.101153,0.092357,0.118745,0.109949,0.105551,0.272673,0.290265
1850,0.029041,0.067763,0.048402,0.067763,0.082283,0.091964,0.116165,-0.140365,0.140365,0.159726,...,0.096804,0.116165,0.140365,0.087123,0.082283,0.198448,0.179087,0.169407,0.353334,0.372695
1851,0.041236,0.057730,0.049483,0.037112,0.061854,0.065977,0.082472,-0.214426,0.090719,0.094842,...,0.090719,0.115460,0.144325,0.086595,0.078348,0.185561,0.160820,0.148449,0.379369,0.399987


In [22]:
data_y = pd.DataFrame(Norm_Y)
data_y

,0,1,2,3,4,5,6,7,8,9,...,458,459,460,461,462,463,464,465,466,467
0,0.095992,0.004174,0.020868,-0.137727,-0.029215,-0.087645,-0.242066,-0.346405,-0.354752,-0.404835,...,0.008347,-0.004174,0.000000,0.012521,0.016694,-0.283802,-0.271281,-0.254587,-0.308843,-0.325537
1,0.098158,-0.036809,0.012270,-0.165641,-0.079753,-0.128832,-0.257664,-0.288338,-0.355822,-0.404900,...,-0.018405,-0.024539,0.012270,-0.012270,0.000000,-0.269934,-0.257664,-0.251529,-0.288338,-0.306743
2,0.088218,-0.009802,0.009802,-0.147031,-0.049010,-0.107822,-0.245051,-0.264655,-0.333269,-0.382279,...,-0.009802,-0.019604,-0.019604,0.000000,0.000000,-0.264655,-0.254853,-0.245051,-0.294061,-0.303863
3,0.078045,-0.009756,0.000000,-0.151212,-0.048778,-0.107312,-0.268280,-0.258524,-0.385348,-0.439003,...,-0.024389,-0.043900,-0.058534,-0.014633,-0.014633,-0.346325,-0.326814,-0.307302,-0.443881,-0.468270
4,0.092146,-0.035103,0.004388,-0.157965,-0.074594,-0.127249,-0.254499,-0.215007,-0.329093,-0.372972,...,-0.030715,-0.048267,-0.030715,-0.026327,-0.017552,-0.280826,-0.267662,-0.258886,-0.337869,-0.346645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,0.103515,-0.018821,0.009410,-0.155272,-0.056463,-0.112925,-0.258787,-0.301134,-0.367007,-0.423470,...,-0.014116,-0.028231,-0.014116,-0.009410,-0.004705,-0.291724,-0.277608,-0.268197,-0.329365,-0.338776
1849,0.096755,-0.013194,0.013194,-0.153928,-0.048377,-0.105551,-0.259479,-0.321051,-0.373826,-0.430999,...,-0.008796,-0.021990,-0.013194,0.000000,0.000000,-0.294663,-0.277071,-0.268275,-0.321051,-0.338642
1850,0.096804,-0.019361,0.019361,-0.150046,-0.058082,-0.106484,-0.227489,-0.314612,-0.324293,-0.372695,...,0.004840,-0.004840,0.024201,0.004840,0.014521,-0.217809,-0.212968,-0.208128,-0.193608,-0.208128
1851,0.094842,-0.008247,0.016494,-0.152572,-0.045359,-0.103089,-0.251538,-0.334010,-0.362875,-0.420605,...,0.000000,-0.008247,0.000000,0.004124,0.008247,-0.280403,-0.268033,-0.255662,-0.296898,-0.309268


In [23]:
df = pd.concat([data_x,data_y],axis=1)
df

,0,1,2,3,4,5,6,7,8,9,...,458,459,460,461,462,463,464,465,466,467
0,0.041736,0.050083,0.050083,0.029215,0.050083,0.058430,0.075124,-0.187810,0.083471,0.087645,...,0.008347,-0.004174,0.000000,0.012521,0.016694,-0.283802,-0.271281,-0.254587,-0.308843,-0.325537
1,0.000000,-0.067483,0.018405,-0.012270,-0.067483,-0.036809,0.061349,0.061349,0.092023,0.085888,...,-0.018405,-0.024539,0.012270,-0.012270,0.000000,-0.269934,-0.257664,-0.251529,-0.288338,-0.306743
2,0.058812,0.068614,0.058812,0.019604,0.068614,0.068614,0.049010,-0.254853,0.039208,0.039208,...,-0.009802,-0.019604,-0.019604,0.000000,0.000000,-0.264655,-0.254853,-0.245051,-0.294061,-0.303863
3,0.078045,0.004878,0.048778,-0.053656,-0.014633,-0.024389,-0.034145,-0.234135,-0.068289,-0.087801,...,-0.024389,-0.043900,-0.058534,-0.014633,-0.014633,-0.346325,-0.326814,-0.307302,-0.443881,-0.468270
4,0.065819,0.030715,0.043879,-0.017552,0.021940,0.013164,-0.004388,-0.232559,-0.017552,-0.026327,...,-0.030715,-0.048267,-0.030715,-0.026327,-0.017552,-0.280826,-0.267662,-0.258886,-0.337869,-0.346645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,0.042347,0.079989,0.051757,0.037642,0.084694,0.079989,0.056463,-0.282313,0.051757,0.056463,...,-0.014116,-0.028231,-0.014116,-0.009410,-0.004705,-0.291724,-0.277608,-0.268197,-0.329365,-0.338776
1849,0.043980,0.087959,0.057173,0.048377,0.092357,0.092357,0.070367,-0.241887,0.070367,0.074765,...,-0.008796,-0.021990,-0.013194,0.000000,0.000000,-0.294663,-0.277071,-0.268275,-0.321051,-0.338642
1850,0.029041,0.067763,0.048402,0.067763,0.082283,0.091964,0.116165,-0.140365,0.140365,0.159726,...,0.004840,-0.004840,0.024201,0.004840,0.014521,-0.217809,-0.212968,-0.208128,-0.193608,-0.208128
1851,0.041236,0.057730,0.049483,0.037112,0.061854,0.065977,0.082472,-0.214426,0.090719,0.094842,...,0.000000,-0.008247,0.000000,0.004124,0.008247,-0.280403,-0.268033,-0.255662,-0.296898,-0.309268


In [24]:
df['pitch'] = pitch
df['yaw'] = yaw
df['roll'] = roll
df

,0,1,2,3,4,5,6,7,8,9,...,461,462,463,464,465,466,467,pitch,yaw,roll
0,0.041736,0.050083,0.050083,0.029215,0.050083,0.058430,0.075124,-0.187810,0.083471,0.087645,...,0.012521,0.016694,-0.283802,-0.271281,-0.254587,-0.308843,-0.325537,-0.399231,0.018227,0.085676
1,0.000000,-0.067483,0.018405,-0.012270,-0.067483,-0.036809,0.061349,0.061349,0.092023,0.085888,...,-0.012270,0.000000,-0.269934,-0.257664,-0.251529,-0.288338,-0.306743,0.470065,1.189533,0.300959
2,0.058812,0.068614,0.058812,0.019604,0.068614,0.068614,0.049010,-0.254853,0.039208,0.039208,...,0.000000,0.000000,-0.264655,-0.254853,-0.245051,-0.294061,-0.303863,-0.184650,0.881137,-0.236852
3,0.078045,0.004878,0.048778,-0.053656,-0.014633,-0.024389,-0.034145,-0.234135,-0.068289,-0.087801,...,-0.014633,-0.014633,-0.346325,-0.326814,-0.307302,-0.443881,-0.468270,-0.175379,0.299208,-0.373374
4,0.065819,0.030715,0.043879,-0.017552,0.021940,0.013164,-0.004388,-0.232559,-0.017552,-0.026327,...,-0.026327,-0.017552,-0.280826,-0.267662,-0.258886,-0.337869,-0.346645,-0.026812,0.011965,-0.220662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,0.042347,0.079989,0.051757,0.037642,0.084694,0.079989,0.056463,-0.282313,0.051757,0.056463,...,-0.009410,-0.004705,-0.291724,-0.277608,-0.268197,-0.329365,-0.338776,-0.306358,-0.283822,0.038554
1849,0.043980,0.087959,0.057173,0.048377,0.092357,0.092357,0.070367,-0.241887,0.070367,0.074765,...,0.000000,0.000000,-0.294663,-0.277071,-0.268275,-0.321051,-0.338642,-0.367547,-0.429723,0.122791
1850,0.029041,0.067763,0.048402,0.067763,0.082283,0.091964,0.116165,-0.140365,0.140365,0.159726,...,0.004840,0.014521,-0.217809,-0.212968,-0.208128,-0.193608,-0.208128,-0.156035,0.567114,-0.108536
1851,0.041236,0.057730,0.049483,0.037112,0.061854,0.065977,0.082472,-0.214426,0.090719,0.094842,...,0.004124,0.008247,-0.280403,-0.268033,-0.255662,-0.296898,-0.309268,-0.197102,-0.070430,0.105118


### 7.Shuffle and save dataset

In [40]:
from sklearn.utils import shuffle
df = shuffle(df)
df

,0,1,2,3,4,5,6,7,8,9,...,461,462,463,464,465,466,467,pitch,yaw,roll
1838,0.067040,0.113968,0.040224,0.006704,0.113968,0.073744,-0.046928,-0.382129,-0.100560,-0.107264,...,-0.067040,-0.046928,-0.288273,-0.268161,-0.261457,-0.328497,-0.348609,0.293546,-1.270909,-0.213710
775,0.035654,0.106963,0.049025,0.044568,0.111420,0.098049,0.040111,-0.285234,0.026741,0.031198,...,-0.017827,-0.013370,-0.285234,-0.271864,-0.258494,-0.316432,-0.334259,-0.086412,-0.555462,-0.004961
680,0.050533,0.106681,0.050533,0.044918,0.112296,0.095451,0.033689,-0.291969,0.016844,0.022459,...,-0.011230,-0.005615,-0.280739,-0.263895,-0.258280,-0.291969,-0.308813,0.006828,-0.563913,0.054653
1248,0.050731,-0.016910,0.039457,-0.011274,-0.022547,-0.005637,0.045094,-0.073278,0.056368,0.050731,...,-0.016910,-0.005637,-0.259292,-0.248018,-0.236745,-0.276202,-0.298749,0.032601,0.657255,-0.001972
130,0.027320,-0.040980,0.027320,-0.027320,-0.047810,-0.034150,0.034150,-0.040980,0.040980,0.034150,...,-0.013660,-0.006830,-0.273198,-0.259538,-0.252708,-0.286858,-0.300518,-0.398765,1.033832,-0.175923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523,0.041029,0.069749,0.049235,0.036926,0.073852,0.073852,0.069749,-0.246173,0.077955,0.082058,...,-0.012309,-0.004103,-0.250275,-0.242070,-0.237967,-0.258481,-0.274893,-0.007852,-0.097847,0.073207
895,0.016641,-0.066564,0.022188,-0.027735,-0.072111,-0.044376,0.049923,0.000000,0.077658,0.077658,...,-0.022188,-0.011094,-0.255162,-0.244068,-0.238521,-0.255162,-0.288444,-0.147490,1.048776,0.005515
1629,0.101092,0.022975,0.055141,-0.078117,-0.004595,-0.032166,-0.096497,-0.307871,-0.151638,-0.179209,...,-0.055141,-0.050546,-0.321656,-0.298681,-0.284896,-0.450319,-0.473294,-0.044172,0.177742,-0.673360
1006,0.042903,0.081040,0.057205,0.047670,0.085807,0.085807,0.071506,-0.233585,0.071506,0.076273,...,0.009534,0.009534,-0.262187,-0.252653,-0.238352,-0.281255,-0.286023,-0.170808,-0.234884,0.075731


In [41]:
pd.DataFrame.to_csv(df,"dataset.csv", index=False)

----

In [42]:
dataset = pd.read_csv("dataset.csv")
dataset

,0,1,2,3,4,5,6,7,8,9,...,461.1,462.1,463.1,464.1,465.1,466.1,467.1,pitch,yaw,roll
0,0.067040,0.113968,0.040224,0.006704,0.113968,0.073744,-0.046928,-0.382129,-0.100560,-0.107264,...,-0.067040,-0.046928,-0.288273,-0.268161,-0.261457,-0.328497,-0.348609,0.293546,-1.270909,-0.213710
1,0.035654,0.106963,0.049025,0.044568,0.111420,0.098049,0.040111,-0.285234,0.026741,0.031198,...,-0.017827,-0.013370,-0.285234,-0.271864,-0.258494,-0.316432,-0.334259,-0.086412,-0.555462,-0.004961
2,0.050533,0.106681,0.050533,0.044918,0.112296,0.095451,0.033689,-0.291969,0.016844,0.022459,...,-0.011230,-0.005615,-0.280739,-0.263895,-0.258280,-0.291969,-0.308813,0.006828,-0.563913,0.054653
3,0.050731,-0.016910,0.039457,-0.011274,-0.022547,-0.005637,0.045094,-0.073278,0.056368,0.050731,...,-0.016910,-0.005637,-0.259292,-0.248018,-0.236745,-0.276202,-0.298749,0.032601,0.657255,-0.001972
4,0.027320,-0.040980,0.027320,-0.027320,-0.047810,-0.034150,0.034150,-0.040980,0.040980,0.034150,...,-0.013660,-0.006830,-0.273198,-0.259538,-0.252708,-0.286858,-0.300518,-0.398765,1.033832,-0.175923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1848,0.041029,0.069749,0.049235,0.036926,0.073852,0.073852,0.069749,-0.246173,0.077955,0.082058,...,-0.012309,-0.004103,-0.250275,-0.242070,-0.237967,-0.258481,-0.274893,-0.007852,-0.097847,0.073207
1849,0.016641,-0.066564,0.022188,-0.027735,-0.072111,-0.044376,0.049923,0.000000,0.077658,0.077658,...,-0.022188,-0.011094,-0.255162,-0.244068,-0.238521,-0.255162,-0.288444,-0.147490,1.048776,0.005515
1850,0.101092,0.022975,0.055141,-0.078117,-0.004595,-0.032166,-0.096497,-0.307871,-0.151638,-0.179209,...,-0.055141,-0.050546,-0.321656,-0.298681,-0.284896,-0.450319,-0.473294,-0.044172,0.177742,-0.673360
1851,0.042903,0.081040,0.057205,0.047670,0.085807,0.085807,0.071506,-0.233585,0.071506,0.076273,...,0.009534,0.009534,-0.262187,-0.252653,-0.238352,-0.281255,-0.286023,-0.170808,-0.234884,0.075731


###  8.Draw axis , get points

In [26]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width / 2
        tdy = height / 2

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),3)

    return img

In [27]:
def get_points_test(image):
    x_points = []
    y_points = []

    faceModule = mediapipe.solutions.face_mesh
    with faceModule.FaceMesh(static_image_mode=True) as faces:

        results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.multi_face_landmarks != None:

            for face in results.multi_face_landmarks:
                x_list=[]
                y_list=[]
                for landmark in face.landmark:
                    x = landmark.x
                    y = landmark.y

                    shape = image.shape 
                    relative_x = int(x * shape[1])
                    relative_y = int(y * shape[0])

                    cv2.circle(image, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)
                    x_list.append(relative_x)
                    y_list.append(relative_y)
                x_points.append(x_list)
                y_points.append(y_list) 

          
  
    return x_points , y_points


In [28]:
def get_df_test(x_points , y_points):
    Center_X = x_points - x_points[:,99].reshape(-1,1)
    Center_Y = y_points - y_points[:,99].reshape(-1,1)

    # normalizing the data to be in the same scale by dividing over the distance between point 10 and point 171
    X_171 = x_points[:,152]
    X_10 = x_points[:,10]
    Y_171 = y_points[:,152]
    Y_10 = y_points[:,10]
    # computing the distance
    distance = np.linalg.norm(np.array((X_10,Y_10)) - np.array((X_171,Y_171)),axis = 0).reshape(-1,1)
    Norm_X = Center_X / distance
    Norm_Y = Center_Y / distance


    data_x = pd.DataFrame(Norm_X)
    data_x
    
    data_y = pd.DataFrame(Norm_Y)

    df = pd.concat([data_x,data_y],axis=1)

    return df

### 9.Split Data

In [13]:
X = dataset.iloc[:,0:936]
y = dataset.iloc[:,936:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [29]:
# X = df.iloc[:,0:936]
# y = df.iloc[:,936:]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### 10. Models

In [30]:
randomForest_pitch = RandomForestRegressor()
randomForest_pitch.fit(X_train, y_train['pitch'])
pred_train = randomForest_pitch.predict(X_train)
pred_test = randomForest_pitch.predict(X_test)
print("train error",mean_absolute_error(y_train['pitch'],pred_train))
print("test error: ",mean_absolute_error(y_test['pitch'],pred_test))


train error 0.043315962744324574
test error:  0.13816462936304152


In [27]:
randomForest_yaw = RandomForestRegressor()
randomForest_yaw.fit(X_train, y_train['yaw'])
pred_train = randomForest_yaw.predict(X_train)
pred_test = randomForest_yaw.predict(X_test)
print("train error",mean_absolute_error(y_train['pitch'],pred_train))
print("test error: ",mean_absolute_error(y_test['pitch'],pred_test))


train error 0.46824842900768593
test error:  0.4869883381640367


In [28]:
randomForest_roll = RandomForestRegressor()
randomForest_roll.fit(X_train, y_train['roll'])
pred_train = randomForest_roll.predict(X_train)
pred_test = randomForest_roll.predict(X_test)
print("train error",mean_absolute_error(y_train['pitch'],pred_train))
print("test error: ",mean_absolute_error(y_test['pitch'],pred_test))


train error 0.32362935473448323
test error:  0.2923525884084788


### 11.Test on video

In [31]:
mp_faceModule = mp.solutions.face_mesh
face_mesh = mp_faceModule.FaceMesh(static_image_mode=True)
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles



In [36]:
cap = cv2.VideoCapture('../video_before.mp4')
frames = []

while (cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break
    height, width, layers = frame.shape
    size = (width, height)
    
    x,y = get_points_test(frame)
    x = np.array(x)
    y = np.array(y)
    nose_x = x[:,99]
    nose_y = y[:,99]    
    points = get_df_test(x , y)

    if points.empty:
        continue
    yaw = randomForest_yaw.predict(points)
    pitch = randomForest_pitch.predict(points)
    roll = randomForest_roll.predict(points)

    out_frame = draw_axis(frame, pitch,yaw,roll,nose_x,nose_y)

    frames.append(out_frame)

In [37]:
video_name = 'Head_Pose_Estimation_finaaaaal.mp4'

video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 15, size)

for frame in frames:
    video.write(frame)

video.release()
cap.release()
cv2.destroyAllWindows()

In [38]:
cap = cv2.VideoCapture('../test2.mp4')
frames = []

while (cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break
    height, width, layers = frame.shape
    size = (width, height)
    
    x,y = get_points_test(frame)
    x = np.array(x)
    y = np.array(y)
    nose_x = x[:,99]
    nose_y = y[:,99]    
    points = get_df_test(x , y)

    if points.empty:
        continue
    yaw = randomForest_yaw.predict(points)
    pitch = randomForest_pitch.predict(points)
    roll = randomForest_roll.predict(points)

    out_frame = draw_axis(frame, pitch,yaw,roll,nose_x,nose_y)

    frames.append(out_frame)

In [39]:
video_name = 'try2.mp4'

video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), 15, size)

for frame in frames:
    video.write(frame)

video.release()
cap.release()
cv2.destroyAllWindows()

### 12.Save 3 Models

In [32]:
with open('randomForest_yaw.pkl', 'wb') as f:
    pickle.dump(randomForest_yaw, f)



In [33]:
with open('randomForest_roll.pkl', 'wb') as f:
    pickle.dump(randomForest_roll, f)


In [34]:
with open('randomForest_pitch.pkl', 'wb') as f:
    pickle.dump(randomForest_pitch, f)


### 13.Load Models

In [33]:
with open('randomForest_yaw.pkl', 'rb') as f:
    randomForest_yaw = pickle.load(f)

In [34]:
with open('randomForest_roll.pkl', 'rb') as f:
    randomForest_roll = pickle.load(f)

In [35]:
with open('randomForest_pitch.pkl', 'rb') as f:
    randomForest_pitch = pickle.load(f)